In [1]:
!pip install streamlit
! pip install pyngrok
!pip install colab-env --upgrade
!pip install git+git://github.com/geopandas/geopandas.git --quiet

     |████████████████████████████████| 8.0 MB 4.1 MB/s 
     |████████████████████████████████| 170 kB 36.7 MB/s 
     |████████████████████████████████| 111 kB 51.0 MB/s 
     |████████████████████████████████| 75 kB 3.2 MB/s 
     |████████████████████████████████| 4.2 MB 40.7 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 122 kB 46.4 MB/s 
     |████████████████████████████████| 786 kB 41.8 MB/s 
     |████████████████████████████████| 368 kB 44.4 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13479 sha256=98a03d6313dab8b33480ab0faa91e5a2cd5ab56026a94fc754063f2da56ae2ce
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting

     |████████████████████████████████| 745 kB 4.2 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.5-py3-none-any.whl size=19262 sha256=22253dd2e3040bdc2ee2b8a2b2d8fb62132c5440f009bd895b5404b9db63afed
  Stored in directory: /root/.cache/pip/wheels/1b/f7/72/35c95a53d15b91dd00df6cf1304d49a31ec5ed6f954c2d4e32
Successfully built pyngrok
  Created wheel for colab-env: filename=colab_env-0.2.0-py3-none-any.whl size=3837 sha256=2c70bb1fd9d30fb3ccce3aacd4c7ea11e37008b74e86c05c350208acadc83d10
  Stored in directory: /root/.cache/pip/wheels/bb/ca/e8/3d25b6abb4ac719ecb9e837bb75f2a9b980430005fb12a9107
Successfully built colab-env
     |████████████████████████████████| 15.4 MB 1.2 MB/s 
     |████████████████████████████████| 6.6 MB 26.5 MB/s 


In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [11]:
%%writefile app.py
import streamlit as st
import pandas as pd
import geopandas as gpd
import geopy
import folium 
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import HuberRegressor, LinearRegression,BayesianRidge,Ridge
from geopy.geocoders import Nominatim
from geopandas.tools import geocode
from shapely.geometry import Point, Polygon

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from PIL import Image


def main():
  #st.title("MODELO")
  menu = ["DEMO",'Reto y Metodología utilizada','Sobre nosotras']
  imageNacho = Image.open('logo.png')
  st.sidebar.image(imageNacho)
  choice = st.sidebar.radio("Menu",menu)
  if choice =='DEMO':
    st.title("""**APLICACIÓN DE PREDICCIÓN DE PRECIO DE LA VIVIENDA NO HORIZONTAL EN BOGOTÁ**""")
    imageBogota = Image.open('Bogota.jpg')
    st.image(imageBogota)
    st.sidebar.title("""**RETO**""")
    st.sidebar.write("* Reto: crear un algoritmo que pueda valorar inmuebles EN BOGOTÁ que no hagan parte de una propiedad horizontal.")

  
  
    st.write('Ya sea que desee comprar, vender, refinanciar o incluso remodelar una casa,  es necesario conocer el precio de su vivienda para tomar una decisión más informada.Por esto se crea una interfaz fácil de usar la cual  ofrece un  margen de error bajo para el precio de hogares en Bogotá, tomando en consideración estimaciones del catastro de Bogotá que incluye información relevante del sector(cercania a parques, centros comerciales, escuelas, zonas industriales, etc) que pueden inclementar o devaluar el precio de una vivienda.')
    st.write('* Ingrese los datos de su vivienda,  responda 1 si la respuesta es afirmativa y 0 en otro caso')

    DatosFinales = pd.read_csv('/content/gdrive/MyDrive/shp/DatosFinales.csv')
    X = pd.DataFrame(DatosFinales.iloc[:,2:27])
    Y = pd.DataFrame(DatosFinales, columns=["y"])
   
    
    # Sidebar
    # Header of Specify Input Parameters
   # st.sidebar.header('Paarámetros de entrada')
    col1,col2=st.beta_columns(2)

    def user_input_features():
      area = st.slider('¿Cuál es el área de tu inmueble en metros cuadrados?',min_value=65,step=1,format='%d',max_value=500)
      ascensor = col1.radio('¿Cuenta con ascensor?',('1','0'))
      balcon = col2.radio('¿Cuenta con balcon?',('1','0'))
      conjuntocerrado = col1.radio('¿El inmueble se enecuentra en un conjunto cerrado?',('1','0'))
      cuartoservicio = col2.radio('¿El inmueble cuenta con cuarto de servicio?',('1','0'))
      deposito =col1.radio('¿El inmueble cuenta con deposito?',('1','0'))
      estudio = col2.radio('¿El inmueble cuenta con estudio?',('1','0'))
      garajes = col1.radio('¿El inmueble cuenta con garaje?',('1','0'))
      gimnasio= col2.radio('¿El inmueble cuenta con gimnasio?',('1','0'))
      halldealcobas =  col1.radio('¿El inmueble cuenta con vestibulo?',('1','0'))
      parqueaderovisitantes = col2.radio('¿El inmueble cuenta con parqueadero para visitantes?',('1','0'))
      piscina =  col1.radio('¿El inmueble cuenta con piscina?',('1','0'))
      vigilancia = col2.radio('¿El inmueble cuenta con vigilancia?',('1','0'))
      porteria = col1.radio('¿El inmueble cuenta con porteria?',('1','0'))
      remodelado = col2.radio('¿El inmueble esta remodelado?',('1','0'))
      terraza = col1.radio('¿El inmueble cuenta con terraza?',('1','0'))
      piso = col2.selectbox('¿Con cuentos pisos cuenta el inmueble?',('1','2','3','4','5','6'))
      estrato =col1.selectbox('¿Que estrato es?',('1','2','3','4','5','6'))
      habitaciones = col2.selectbox('¿Con cuantas habitaciones cuenta el inmueble?',('1','2','3','4','5','6'))
      valoradministracion = col1.number_input('¿Cuál es el valor de administracion? , ejemplo: $500.000')
      valorventa = col2.number_input('¿Cual cree es el valor total de su inmueble?' )
      banos = col1.number_input('¿Con cuantos baños cuenta el inmueble ?',min_value=0,step=1,format='%d',max_value=6)
      banoservicio = col2.number_input('¿Con cuantos baños de servicio cuenta el inmueble ?',min_value=0,step=1,format='%d',max_value=3)

      country = st.text_input("Pais", "Colombia")
      city = st.text_input("Ciudad", "Bogota")
      province = st.text_input("Barrio", "Kennedy")
      street = st.text_input("Direccián", "Transversal 78  # 41-48")
      geolocator = Nominatim(user_agent="app")
      geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
      location = geolocator.geocode(street+", "+city+", "+province+", "+country)
    
      latitud = location.latitude
      longitud = location.longitude
      map_data = pd.DataFrame({'lat': [latitud], 'lon': [longitud]})
      st.map(map_data)
    
      data = {'area': area,
              'ascensor': ascensor,
              'balcon': balcon,
              'banos': banos,
              'banoservicio':banoservicio,
              'conjuntocerrado':conjuntocerrado,
              'cuartoservicio':cuartoservicio,
              'deposito':deposito,
              'estrato':estrato,
              'estudio':estudio,
              'garajes':garajes,
              'gimnasio':gimnasio,
              'habitaciones':habitaciones,
              'halldealcobas':halldealcobas,
              'parqueaderovisitantes':parqueaderovisitantes,
              'piscina':piscina,
              'piso':piso,
              'porteria':porteria,
              'remodelado':remodelado,
              'terraza':terraza,
              'vigilancia':vigilancia,
              'valoradministracion':valoradministracion,
              'valorventa':valorventa,
              'latitud':latitud,
              'longitud':longitud}

      features = pd.DataFrame(data, index=[0])
      return features

    df = user_input_features()

    st.header('Parámetros de entrada.')
    st.write(df)

    model = RandomForestRegressor()
    model.fit(X, Y)
    #Predicciones
    prediction = model.predict(df)

    st.header('El valor por metro cuadrado de tu inmueble es: ')
    st.write(prediction)
    
#####################################################################################################################################
  elif choice =='Reto y Metodología utilizada':
    st.title("""**RETO**""")
    st.write("* Reto: crear un algoritmo que pueda valorar inmuebles EN BOGOTÁ que no hagan parte de una propiedad horizontal.")
    st.write('Todo surge de la necesitad de la compañia Habi, única plataforma digital en Colombia que compra inmuebles en 10* días, los remodela y los vuelve a sacar a la venta a un precio competitivo. Para hacerlo, esta proptech recopila desde diversas fuentes toda la oferta vigente de inmuebles en venta, la ordena y la clasifica para posteriormente compararla entre sí y establecer precios. Los expertos de Habi perfeccionan todos los días el algoritmo con el que identifican el valor de los inmuebles pertenecientes a una propiedad horizontal. Sin embargo, los inmuebles que NO hacen parte de una propiedad horizontal aún representan un reto, ya que su valorización depende de determinantes que no considera correctamente su algoritmo actual. Es po esto que  Habi junto con Change lanzarón un reto denominado ** El algoritmo es correcto** .')

  elif choice =='Sobre nosotras':
    st.title(""" **EQUIPO**""")
    col1, col2 = st.beta_columns(2)
    
    col1.markdown('### Sara Lucia Acosta Pinzón')
    imageSara = Image.open('Sara_Acosta.jpeg')
    col1.image(imageSara)
    col1.markdown('Estudiante de Maestria en Estadística  ** Universidad Nacional de Colombia** slacostap@unal.edu.co')
    col1.markdown( '[LinkedIn](https://www.linkedin.com/in/saraluacosta/)')
    st.subheader("Otro titulo")

    col2.markdown('### Valeria Bejarano Salcedo')
    imageValeria = Image.open('Valeria_Bejarano.png')
    col2.image(imageValeria)
    col2.markdown('Estudiante de Maestria en Estadística  ** Universidad Nacional de Colombia** vbejaranos@unal.edu.co')
    col2.markdown( '[LinkedIn](https://www.linkedin.com/in/valeria-bejarano-salcedo-a31600176/)')

if __name__ == '__main__':
	main()

Overwriting app.py


In [4]:
! ngrok authtoken 1vVf8QwOLu8NpideEvK907dbPm8_3sgEvmkQ4C8x2Jdu3aipd

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [5]:
from pyngrok import ngrok

In [6]:
!streamlit run --server.port 80 app.py &>/dev/null &

In [7]:
public_url = ngrok.connect(port='80')

In [8]:
public_url

<NgrokTunnel: "http://c7a6bd40626a.ngrok.io" -> "http://localhost:80">

In [ ]:
ngrok.kill()